In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from tqdm import tqdm
from mlpckg.stat_method import *
from mlpckg.ML_method import *
from mlpckg.util_method import *
from mlpckg.final_var_select import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# select_key() argument ("1.novem/ 2.trans/ 3.bipun/ 4.paren ")
# select_key() return (target_species,key)

num = 1
target_species,key= select_key(num)


ratio = 0

In [ ]:
#csv for model
df_train = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_train_minmax.csv")

df_train = df_train.drop(df_train[(df_train[target_species + str('_18')]!=0)&(df_train['Species']!= target_species)].index).reset_index(drop=True)

In [ ]:
#remain columns which you need
delivered_package = set(df_train.columns.tolist()) - set(['Source','State','Year','Latitude','Longitude','public_positional_accuracy', 
                                                    str(target_species+str('_18'))])

delivered_package = list(delivered_package)

df_train = df_train.sort_values(by='Species').reset_index(drop=True)
df_train_18 = df_train[delivered_package]


# target 1: presence, 0: absence
df_train_18['target'] = 0

df_train_18.loc[df_train_18.Species==target_species,'target'] = 1

df_train_18 = df_train_18.sort_values(by='target', ascending=False).reset_index(drop=True)

In [ ]:
df_train_18.rename(columns={"Species":"Species_18"},inplace=True)

### 1.통계분석(독립회귀분석 -> 다중회귀분석)을 통한 변수 선정

In [ ]:
df_train_copy = df_train_18.copy()
pot = custom_linreg(df_train_copy,target_species)

In [ ]:
# 1) linear regress result
lemon=[]
for i in pot:
    lemon.append(i.replace(' ','_').replace('-',''))

    
lemon2 = ' + '.join(lemon)
lemon
lemon2

for i in df_train_copy.columns:
    for j in range(0, len(pot)):
        if i == pot[j]:
            df_train_copy.rename(columns={i: lemon[j]}, inplace = True)

In [ ]:
# linear regress result -> multiple regress 
lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy,viewoption=True)[1]

# onemore
lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy)[1]
#onemore
lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy)[1]


res, lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy)

In [ ]:
#linear regress result -> multiple regress result
green_moss = pd.DataFrame(res.pvalues)
green_moss = green_moss[green_moss[0] < 0.05]
green_moss.drop(['Intercept'], inplace=True)

add_p1 = list(green_moss.index)
add_p1

### 2.통계분석(다중회귀분석)을 통한 변수 선정

In [ ]:
df_train_copy2 = df_train_18.copy()

jazz = list(df_train_copy2.columns)
jazz.remove('target')
jazz.remove('Species_18')

# 2) only multiple regress
lemon=[]
for i in jazz:
    lemon.append(i.replace(' ','_').replace('-',''))
print(len(lemon))  

rime = ' + '.join(lemon)
print(len(rime.split("+")))

In [ ]:
col_index=[]
for cols in df_train_copy2.columns:
    col_index.append(cols.replace(' ','_').replace('-',''))

df_train_copy2.columns = col_index


In [ ]:
#multiple regress (related Species)
lemon2 = custom_mulreg(olsarg =rime,df=df_train_copy2,viewoption=True)[1]


# multiple regress (related Species)
lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy2)[1]

#multiple regress (related Species)
lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy2)[1]

# multiple regress (related Species)
res, lemon2 = custom_mulreg(olsarg =lemon2,df=df_train_copy2)[0], custom_mulreg(olsarg =lemon2,df=df_train_copy2,viewoption=True)[1]

In [ ]:
#multiple regress result(related Species)
green_moss = pd.DataFrame(res.pvalues)
green_moss = green_moss[green_moss[0] < 0.05]
green_moss.drop(['Intercept'], inplace=True)

add_p2 = list(green_moss.index)
add_p2

In [ ]:
df_train_copy = df_train_copy.drop(columns = 'target')
df_train_copy = df_train_copy.drop(columns = 'Species_18')

In [ ]:
# 단순히 많이 찍힌 종들이 어떤 종인지 한번 봐보는 용도
# species that were dominantly recorded around the targets were checked
dicts2 = {}
for i in df_train_copy.columns:
    dicts2[i] = len(df_train_copy[df_train_copy[i] != 0])
    
dicts2 = sorted(dicts2.items(), key = lambda item: item[1], reverse = True)
dicts2 = dicts2[:4]
add_f = [x[0] for x in dicts2]

#replace ' ' to '_'
add_f2 = []
for i in add_f:
    add_f2.append(i.replace(' ', '_'))
    
add_f = add_f2


In [ ]:
MC = list(set(add_p1 + add_f))
DJ = list(set(add_p2 + add_f))


#MC는 add_p1은 c7이 빠지게됨.
print(len(MC) == len(add_p1))
print(len(DJ) == len(add_p2))

### 3.다중공선성을 고려한 변수 압축
- 10 이하, 엄격할 경우 5 이하

In [ ]:
MC = custom_vif_varlist(df=df_train_copy2,varlist=MC)
DJ = custom_vif_varlist(df=df_train_copy2,varlist=DJ)

print(len(MC))
print(len(DJ))

In [ ]:
quiz = len(df_train_copy2[df_train_copy2.Species_18 == target_species].index)
quiz

### 4. ML 결과

In [ ]:
ML(df=df_train_copy2.drop('Species_18',axis=1),
targetnum=quiz,
varlist=DJ,
varlistname='DJ',
target_species=target_species,
bagnum=50,repnum=50,
testsize=0.1,learningrate=0.7,viewoption=False,to_csv=False)

### 5. feature importance , shap index => 변수압축

In [ ]:
# check feature importance: shap values
final_vars= final_var_select(df=df_train_copy2.drop('Species_18',axis=1),
targetnum=quiz,
varlist=DJ,
target_species=target_species,
bagnum=10,repnum=10,
testsize=0.1,learningrate=0.7,returnsize=15,viewoption=False,to_csv=False)

### 6. 5에서 결정한 변수들만 사용했을때, 모델의 성능 평가

In [ ]:
ML(df=df_train_copy2.drop('Species_18',axis=1),
targetnum=quiz,
varlist=final_vars,
varlistname='SHAP',
target_species=target_species,
bagnum=50,repnum=50,
testsize=0.1,learningrate=0.7,viewoption=False,to_csv=False)